# Download

In [ ]:
install.packages("knitr")
install.packages("dplyr")
install.packages("stringr")

### Introduction

The purpose of this document is to show how I downloaded all of the standard rating files from the FIDE organizations website.

### Import libraries

I don't think any packages are necessary to download any of the files. Still though, I use `knitr` and `dplyr` to display a table of urls that we will call on to download from later on.

In [1]:
library(knitr)
library(dplyr)
library(stringr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




### Set up working directory

Before proceeding, we need to set up a working directory to store all of the downloaded files. Make sure to have an empty folder you can dump the files into. I've created a `Downloaded files` subfolder within the folder of this `.Rmd` document.

**IMPORTANT: Please adjust `path` to the path of the folder you want the data to be in.**

If you're not sure how to set your directory, you can choose `path` interactively using `choose.dir()`. If you choose to do so, make sure to comment out the first line and uncomment out the 2nd line.


In [23]:
# path = "~/download"
path = choose.dir()

opts_knit$set(root.dir = path)

### FIDE file format

R's default is to adjust all numbers, beginning with 0, and truncatte the leading zeros. 

Below we setup many useful vectors of strings that will be useful for creating urls that we will download the `.zip` files from.

In [24]:
year_vector <- as.character(1:as.numeric(substr(Sys.Date(), 3, 4)))
month_vector <- tolower(substr(month.name, 1, 3))
url_vector = url_vect_destfile = rep(0, length(month_vector)*length(year_vector))

for(i in 1:length(year_vector)){
  if(nchar(year_vector[i]) == 1){
    year_vector[i] = paste("0", year_vector[i], sep = "")
  }
}

latest <- format(Sys.Date(), format="%b%Y")
latest <- paste(tolower(substr(latest, 1, 1)), substr(latest, 2, 3), substr(latest, nchar(latest)-1, nchar(latest)), "frl.zip", sep = "")

latest

[1] "jan22frl.zip"

`latest` tells us the most recent file FIDE should have hosted on their website.

Below, we put it all together using a somewhat ugly for loop, but it accomplishes our job nicely to create the urls we desire. Note that in August 2014 (which explains the `else if` statement at the 140th iteration), FIDE added an extra word, "standard_" to the URLs to be downloaded from. 

In [25]:
for(i in seq_along(year_vector)){
  for(j in seq_along(month_vector)){
    if(12*(i-1)+j <= 140){
      url_vector[12*(i-1)+j] = paste("http://ratings.fide.com/download/", 
                                     month_vector[j], 
                                     year_vector[i], 
                                     "frl.zip", 
                                     sep = "")
    }
    else if(12*(i-1)+j > 140){
      url_vector[12*(i-1)+j] = paste("http://ratings.fide.com/download/standard_", 
                                     month_vector[j], 
                                     year_vector[i], 
                                     "frl.zip", 
                                     sep = "")
    }
    url_vect_destfile[12*(i-1)+j] <- substr(url_vector[12*(i-1)+j], nchar(url_vector[12*(i-1)+j])- 11 , nchar(url_vector[12*(i-1)+j]))
  }
}

url_vect_destfile <- url_vect_destfile[1:which(url_vect_destfile == latest)]
url_vector <- url_vector[1:which(url_vect_destfile == latest)]

rm(list=setdiff(ls(), c("url_vector", "url_vect_destfile")))

In [26]:
# url_vector <- tail(url_vector,25)
# url_vect_destfile <- tail(url_vect_destfile,25)



### URLs and their files

In [30]:
data.frame(URL = url_vector,
           File = url_vect_destfile)%>%
slice(c(1:5, (n()-4):n()))%>% #first/last 5 observations
kable()



|URL                                                    |File         |
|:------------------------------------------------------|:------------|
|http://ratings.fide.com/download/standard_jan20frl.zip |jan20frl.zip |
|http://ratings.fide.com/download/standard_feb20frl.zip |feb20frl.zip |
|http://ratings.fide.com/download/standard_mar20frl.zip |mar20frl.zip |
|http://ratings.fide.com/download/standard_apr20frl.zip |apr20frl.zip |
|http://ratings.fide.com/download/standard_may20frl.zip |may20frl.zip |
|http://ratings.fide.com/download/standard_sep21frl.zip |sep21frl.zip |
|http://ratings.fide.com/download/standard_oct21frl.zip |oct21frl.zip |
|http://ratings.fide.com/download/standard_nov21frl.zip |nov21frl.zip |
|http://ratings.fide.com/download/standard_dec21frl.zip |dec21frl.zip |
|http://ratings.fide.com/download/standard_jan22frl.zip |jan22frl.zip |

Above is a trimmed table of URLs and their corallary files. This is what what will be inputted in the `download_all()` function down below.

### Download all chess files silently

I'm fairly sure you can remove the two lines involving `old`, but I'd keep them in case you start to get spammed with messages.

`download_all()` quietly checks each URL we will visit and downloads the file present at each URL. If a file doesn't exist at a URL (many of the early year URLs don't), then the function skips over the error.

**Note:** Download times vary!

In [34]:
download_all <- function(link, dest){
  if (!file.exists(dest)) {
    tryCatch({
      download.file(link, dest, method="auto", quiet = TRUE) 
    }, error=function(e){})
  }
}


old <- getOption("warn"); options(warn = -1)
invisible(mapply(download_all, url_vector, url_vect_destfile))
options(warn = old)

### Unzipping and cleaning 

Some brief housecleaning is taken care of below After the step below, you should only have text files in the directory you set at the beginning. All that's done below is unzipping the `.zip` files and deleting the `.zip` files.

In [35]:
invisible(sapply(list.files(pattern = "*.zip"), function(x) unzip(x, exdir = getwd())))

unlink(list.files(pattern = "*.zip"))

Lastly, we can verify what is in our directory.

In [36]:
list.files(pattern = "*.txt")%>%
head()

[1] "standard_apr20frl.txt" "standard_apr21frl.txt" "standard_aug20frl.txt"
[4] "standard_aug21frl.txt" "standard_dec20frl.txt" "standard_dec21frl.txt"

As we can see, we can see the downloaded and unzipped text files: Success!

### Blitz & Rapid data

Likewise, we can repeat the same steps on the blitz and rapid data sets.

### Blitz

In [ ]:
blitz_path = "~/GitHub/FIDE/Chess Scripts/Step 1 - Download/Blitz"

blitz = url_vector[141:length(url_vector)]%>%
        str_replace("standard", "blitz")

blitz_dest = url_vect_destfile[141:length(url_vector)]

opts_knit$set(root.dir = blitz_path)

mapply(download_all, blitz, blitz_dest)

invisible(sapply(list.files(pattern = "*.zip"), function(x) unzip(x, exdir = getwd())))

unlink(list.files(pattern = "*.zip"))

### Rapid

In [ ]:

rapid_path = "~/GitHub/FIDE/Chess Scripts/Step 1 - Download/Rapid"


rapid = url_vector[141:length(url_vector)]%>%
        str_replace("standard", "rapid")

rapid_dest = url_vect_destfile[141:length(url_vector)]

opts_knit$set(root.dir = rapid_path)

mapply(download_all, rapid, rapid_dest)

invisible(sapply(list.files(pattern = "*.zip"), function(x) unzip(x, exdir = getwd())))

unlink(list.files(pattern = "*.zip"))

# Reformat

The purpose of this document is to how to process the text files we have previously seen in `Download.pdf` within the Step 1 folder.

### Libraries

The first steps to succesfully reformatting all of the text files in the `Downloads files` folder is importing all of the necessary libraries.

In [40]:
install.packages("devtools")

package 'devtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\loren\AppData\Local\Temp\RtmpeOoW6p\downloaded_packages


In [45]:
library(knitr)
library(tidyverse)
library(data.table)

library(foreach)
library(doParallel)
library(stringi)
library(lubridate)

A very quick summary of why each package is used:

- `knitr` is imported to set the working directory for the R-Markdown document.
- `tidyverse` is useful for its `%>%` (pipe) operator.
- `data.table` is extremely fast for writing dataframes to stored files.
- `Kmisc` is used for its `readlines()` which the fast version of R's base function, `readLines()` .
- `foreach` and `doParallel` are useful for parallel processing exporting data.frames to csv files

### Directories

In order to work with the data files in Step 1 and rework them, we need to:

1. Assign a path we will import the text files
2. Assign a path we will to store the created .csv files

In [46]:
path = choose.dir()

In [47]:
destination = choose.dir()
opts_knit$set(root.dir = path)

### Function

This is by far the most cumbersome function to read, let alone work through. Long story short, many of the text files are hopelessly formatted:

- Some have improperly labeled columns
- Some have misspelled columns
- Some have their columns out line
- Some have blank & missing rows
- Some don't even have column headers to begin with

I would like to thank Kirsan Ilyumzhinov and his aliens for bestowing the challenge of fixing his organization's publically available files. 

In [48]:

reformat <- function(file_csv, df){

#Reformat October 2002
if(file_csv == "OCT02FRL.TXT"){df[1] <- df[1]%>%
                                           gsub("COUNTRY", "  Fed ", .)%>%
                                           gsub("GAMES", "Gms", .)%>%
                                           gsub("BIRTHDAY", " BIRTHDAY", .)}

#Reformat April 2003
else if(file_csv == "APR03FRL.TXT"){df[1] <- df[1]%>%
                                          gsub("   CODE  ","ID_NUMBER",.)%>%
                                          gsub("COUNTRY","  FED  ",.)%>%
                                          gsub(" APR03", "APR03 ", .)%>%
                                          gsub(" GAMES", "GMS   ", .)%>%
                                          gsub("  BIRTHDAY", "BIRTHDAY  ", .)%>%
                                          gsub(" FLAG", "FLAG ", .)}

#Reformat April, July, October 2004 & January, July 2005
else if(file_csv %in% c("APR04FRL.TXT","JUL04FRL.TXT","OCT04FRL.TXT", 
                        "JAN05FRL.TXT", "JUL05FRL.TXT")){
                                                df[1] <- df[1]%>%
                                                gsub("COUNTRY", "   FED ", . )%>%
                                                gsub("GAMES", "GAME ", .)%>%
                                                gsub(" BIRTHDAY", "BIRTHDAY", .)}

#Reformat January 2006 to July 2012
else if(file_csv == "JAN06FRL.TXT"){df <- df[-2]}
else if(file_csv %in% c("APR06FRL.TXT", "JUL06FRL.TXT", "OCT06FRL.TXT",
                      list.files(pattern = "[0][7-9][Ff][Rr][Ll].[Tt][Xx][Tt]"),
                      list.files(pattern = "[1][0-1][Ff][Rr][Ll].[Tt][Xx][Tt]"),
                      list.files(pattern = "[1][2][Ff][Rr][Ll].[Tt][Xx][Tt]")[1:5])){
                      df[1] <- df[1] %>% gsub("Titl", "Tit ", .)%>%
                                         gsub("Games", "Game ", .)%>%
                                         gsub("July", "Jul", .)}

#Insert underscore to make sure that proper columns are created in the functions afterwards
df[1] <- gsub("ID Number", "ID_NUMBER", df[1])%>%
         gsub("ID number", "ID_NUMBER", .)

return(df)
}


### Indexes

All of the text files lack a proper delimeter to import the data on. Any normal dataset with have a comma or tab delimeter, but these have nothing of the sort. Given this, the function below helps tackle this problem. It grabs the column headers of a given text file and finds the indexes at which we need to insert delimeters at.

In [49]:
indexes <- function(df){ 
column_vector<- df[1]
indexes <- rep(0, nchar(df[1]))
for(i in 1:nchar(column_vector)){
    index = grep("\\s[A-z]", substr(column_vector, i, i+1))
    if (identical(index, integer(0)) == TRUE){indexes[i] = 0}
    else {indexes[i] = 1}
                                }
return(which(indexes == 1))
                      }

### Insert delimeters quickly

As discussed above, we need to insert delimeters at these indexes. Using R's base functions, we can use `utf8ToInt()` and `intToUtf8()`to quickly break down every string into vectors and replace vectors indexes with the delimeter we want. In this case, we will insert a commma. 

In [50]:
utf_func <- function(df, indexed){
string <- utf8ToInt(df)
string[indexed] <- utf8ToInt("*")
return(intToUtf8(string))
}

### File rename

Each exported file needs to be renamed so the function is helpful for that.

In [51]:
filenamer <- function(Year_num) {
  Year_num%>%
  substr(., nchar(.)-11, nchar(.)-7)%>%
  toupper()%>%
  paste(destination, ., ".csv", sep = "")
}

### Write files

Lastly, we need to export the text files using `data.table`'s speedy `fwrite()` function.

In [52]:
fwrite_wrapper <- function(filename, df){
if (file.exists(filename)) {unlink(filename)}
fwrite(list(df), file = filename, quote = FALSE)
}

### All files write

Below is a wrapper function that builds on all of the previously stated functions and puts it all together. Some of the files may have blank lines initially so unfortunately, we have to use `readLines()` (a somewhat slow version of `readlines()`, but allows users to read in files beginnign with blank lines).

In [59]:
All_files_fwrite <- function(year_vector){
text_insert_first = c("jul03frl.txt","OCT03FRL.TXT", "JAN04FRL.TXT", 
                      "APR05FRL.TXT", "jan03frl.txt")
columns = "ID_NUMBER NAME                            TITLE FED  RATING GM  Bday      Flag "

if(year_vector %in% text_insert_first){
df = readLines(year_vector)
  if(nchar(df[1]) != 0) {
    cat("", df, file = year_vector, sep = "\n")
    df <- readLines(year_vector)
    df[1] <- columns} else{df[1] <- columns}
} else {df = readLines(year_vector)}
  
df <- reformat(year_vector, df)
indexed = indexes(df)
df = sapply(df , utf_func, indexed = indexed, USE.NAMES = FALSE)
filename <- filenamer(year_vector)
fwrite_wrapper(df, file = filename)
}


### Multi Processing

The first time I ran a for loop that executes `All_files_fwrite()` on every single text file, it took ~20-25 minutes. The following function runs `All_files_fwrite()` in parallel. It took me a while to put it all together because of a few reasons:

1. `.export = functions` is necessary to import functions from R's global environment into the parallel processing function environment. Otherwise, global functions & variables aren't detected.

2. `detectCores()` is a useful function to check how many cores your computer has. My machine has 8 and it cuts down the original ~20-25 minutes to ~4-5 minutes!

3. `.packages = c("dplyr", "data.table", "Kmisc")` is needed because these packages are heavily involved in many of the functions defined above.

4. The rest is setting up a cluster network, which allows for parallel processing.

In [60]:
multi_processing <- function(Year_num){
                    functions = ls(globalenv())
                    cl <- makeCluster(detectCores()) 
                    clusterExport(cl, functions)
                    registerDoParallel(cl)
                    foreach(i = Year_num,
                            .export = functions,
                            .packages = c("dplyr", 
                                          "data.table")) %dopar% {All_files_fwrite(i)}
                    stopCluster(cl)
                                      }

### A final touch and output

The final chunk here is divided into a few steps:

- `Year_num` gathers all of the text files into a vector.
- `system.time()` captures how long this process takes. If `multi_processing(Year_num)` does not run, you can run the commented out line below it.
- The last line prints out example `.csv` files that been created in the destination directory.


In [61]:
Year_num <- list.files(path = path, pattern = "[Ff][Rr][Ll].[Tt][Xx][Tt]")

system.time(multi_processing(Year_num))

##Run this if parallel processing doesn't work
# system.time(invisible(mapply(All_files_fwrite, Year_num)))

head(list.files(path = destination, pattern = "*.csv"))

  usuário   sistema decorrido 
     0.02      0.04     49.14 

character(0)

We can see the `.csv` files have been created, as a result.

I hope you found this document helpful. Writing this file was extremely time consuming. Many of the functions have only been optimized after several iterations and refinements. 

## Rapid

### Libraries

In [ ]:
library(readr)
library(dplyr)
library(magrittr)
library(stringr)
library(Hmisc)
library(data.table)
library(lubridate)

### Useful functions

In [ ]:
month_conv <- function(year_vector){
for(i in 1:length(year_vector)){
  if(nchar(year_vector[i]) == 1){
    year_vector[i] = paste("0", year_vector[i], sep = "")
  }
}
return(year_vector)
}

first_line <- function(path){
    con <- file(path,"r")
    first_line <- readLines(con,n=1)%>%
                  str_replace("ID Nu", "ID_Nu")
    close(con)
    return(first_line)
}

indexes <- function(column_vector){ 
    indexes <- rep(0, nchar(column_vector))
    for(i in 1:nchar(column_vector)){
        index = grep("\\s[A-z]", substr(column_vector, i, i+1))
        if (identical(index, integer(0)) == TRUE){indexes[i] = 0}
        else {indexes[i] = 1}
                                    }
    return(c(0, which(indexes == 1), nchar(column_vector)))
}

clean <- function(x){
  x[grepl("Fed|FED|COUNTRY", x)] <- "Country"
  x[grepl("Gms|GAMES|GM|Game|GAME", x)] <- "Games"
  x[grepl("K", x)] <- "K_factor"
  x[grepl("FLAG|Flag|flag", x)] <- "Activity"
  x[x %in% c("Wtit","wtit","WTIT", "WTit")] <- "WTitle"
  x[x %in% c("TITLE","Title","title","Tit")] <- "Title"
  x[grepl(string, x)] <- "Rating"
  x[grepl("Born|Age|age|BIRTHDAY|B-day|Bday", x)] <- "Age_Birthday"
  x[grepl("SEX", x)] <- "Sex"
  x[grepl("FOA", x)] <- "FIDE_Online_Arena"
  x[grepl("OTit", x)] <- "Other_Titles"
  return(x)
}

dates <- function(files){
  files%>%
  gsub("rapid_","", ., fixed = TRUE)%>%
  gsub("frl.txt","", ., fixed = TRUE)-> dates
  
  gsub('\\D','', dates)%>%
  paste("20", ., sep = "") -> years
  
  gsub('\\d','', dates)%>%
  capitalize()%>%
  match(month.abb)%>%
  month_conv() -> months
  
  return(paste(years, months, "01", sep = "-"))
}

year_month <- function(x){
paste(month.abb[month(x)]%>%toupper(),
      year(x)-2000, 
      sep = "")
}

### Paths

In [ ]:
path = "~/GitHub/FIDE/Chess Scripts/Step 1 - Download/Rapid/"
files = list.files(path = path)
rapid_paths = paste(path, files, sep = "")
dataset_names = dates(files)
string <- c(month.abb, tolower(month.abb), toupper(month.abb), "RATING")%>%
          paste(collapse = "|")

rapid_paths%>%
sapply(first_line)%>%
sapply(indexes)%>%
sapply(diff)%>%
setNames(files)-> col_positions

rapid_paths%>%
sapply(first_line)%>%
strsplit("\\s+")%>%
lapply(clean)%>%
setNames(files) -> columns

### Import data

In [ ]:
for(i in 1:length(files)){
assign(dataset_names[i], 
       suppressMessages(
         read_fwf(rapid_paths[i], 
                  col_positions = fwf_widths(col_positions[[i]]),
                  skip = 1)%>%
         set_colnames(columns[[i]])
                       )%>%
         mutate(date = dataset_names[i])
      )
}

FIDE <- mget(ls(pattern = "[0-9][0-9]-[0-9][0-9]"))

names(FIDE) = paste("~/GitHub/FIDE/Chess Scripts/Step 2 - Reformat/Rapid/",
              year_month(names(FIDE)), ".csv", sep = "")


### Export data

In [ ]:
for (i in 1:length(FIDE)){
  fwrite(x = FIDE[[i]], 
         file = names(FIDE)[i])
}

## Blitz

### Libraries

In [ ]:
library(readr)
library(dplyr)
library(magrittr)
library(stringr)
library(Hmisc)
library(data.table)
library(lubridate)

### Useful functions

In [ ]:
month_conv <- function(year_vector){
for(i in 1:length(year_vector)){
  if(nchar(year_vector[i]) == 1){
    year_vector[i] = paste("0", year_vector[i], sep = "")
  }
}
return(year_vector)
}

first_line <- function(path){
    con <- file(path,"r")
    first_line <- readLines(con,n=1)%>%
                  str_replace("ID Nu", "ID_Nu")
    close(con)
    return(first_line)
}

indexes <- function(column_vector){ 
    indexes <- rep(0, nchar(column_vector))
    for(i in 1:nchar(column_vector)){
        index = grep("\\s[A-z]", substr(column_vector, i, i+1))
        if (identical(index, integer(0)) == TRUE){indexes[i] = 0}
        else {indexes[i] = 1}
                                    }
    return(c(0, which(indexes == 1), nchar(column_vector)))
}

clean <- function(x){
  x[grepl("Fed|FED|COUNTRY", x)] <- "Country"
  x[grepl("Gms|GAMES|GM|Game|GAME", x)] <- "Games"
  x[grepl("K", x)] <- "K_factor"
  x[grepl("FLAG|Flag|flag", x)] <- "Activity"
  x[x %in% c("Wtit","wtit","WTIT", "WTit")] <- "WTitle"
  x[x %in% c("TITLE","Title","title","Tit")] <- "Title"
  x[grepl(string, x)] <- "Rating"
  x[grepl("Born|Age|age|BIRTHDAY|B-day|Bday", x)] <- "Age_Birthday"
  x[grepl("SEX", x)] <- "Sex"
  x[grepl("FOA", x)] <- "FIDE_Online_Arena"
  x[grepl("OTit", x)] <- "Other_Titles"
  return(x)
}

dates <- function(files){
  files%>%
  gsub("blitz_","", ., fixed = TRUE)%>%
  gsub("frl.txt","", ., fixed = TRUE)-> dates
  
  gsub('\\D','', dates)%>%
  paste("20", ., sep = "") -> years
  
  gsub('\\d','', dates)%>%
  capitalize()%>%
  match(month.abb)%>%
  month_conv() -> months
  
  return(paste(years, months, "01", sep = "-"))
}

year_month <- function(x){
paste(month.abb[month(x)]%>%toupper(),
      year(x)-2000, 
      sep = "")
}

### Paths

In [ ]:
path = "~/GitHub/FIDE/Chess Scripts/Step 1 - Download/Blitz/"
files = list.files(path = path)
Blitz_paths = paste(path, files, sep = "")
dataset_names = dates(files)
string <- c(month.abb, tolower(month.abb), toupper(month.abb), "RATING")%>%
          paste(collapse = "|")

Blitz_paths%>%
sapply(first_line)%>%
sapply(indexes)%>%
sapply(diff)%>%
setNames(files)-> col_positions

Blitz_paths%>%
sapply(first_line)%>%
strsplit("\\s+")%>%
lapply(clean)%>%
setNames(files) -> columns

### Import data

In [ ]:
for(i in 1:length(files)){
assign(dataset_names[i], 
       suppressMessages(
         read_fwf(Blitz_paths[i], 
                  col_positions = fwf_widths(col_positions[[i]]),
                  skip = 1)%>%
         set_colnames(columns[[i]])%>%
         mutate(date = dataset_names[i])
                       )
      )
}

FIDE <- mget(ls(pattern = "[0-9][0-9]-[0-9][0-9]"))

names(FIDE) = paste("~/GitHub/FIDE/Chess Scripts/Step 2 - Reformat/Blitz/",
              year_month(names(FIDE)), ".csv", sep = "")

for (i in 1:length(FIDE)){
  fwrite(x = FIDE[[i]], 
         file = names(FIDE)[i])
}

# Fide Country Codes

This file produces a useful table for us to match country names on. FIDE's Chess olympiad website has these codes which we will export to a .csv file after some webscraping.

In [10]:
import pandas as pd
from bs4 import BeautifulSoup 
import bs4
import urllib.request
import requests
import os

In [2]:
def get_soup_OP(url):
        req = requests.get(url)
        return BeautifulSoup(req.content,'html.parser')  
    
def comment_text(id):
    return(id.text)

def td(soup):
    return(soup.find("td", attrs = {"class": "let"}))

def digit_remove(ls):
    return(''.join([i for i in ls if not i.isdigit()]))
    
def map_list(function, ls):
    return(list(map(function, ls)))

In [3]:
url = "https://www.olimpbase.org/help/help41.html"
soup = get_soup_OP(url)

In [6]:
country_code = map_list(comment_text, soup.find_all("td", attrs = {"class": "ce"}))
country = map_list(digit_remove, map_list(comment_text, map_list(td, soup.find_all("tr")[1:])))
FIDE  = pd.DataFrame([country_code, country]).transpose()
FIDE.columns = ["Code", "Country"]
FIDE.head(10)

,Code,Country
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,AND,Andorra
4,ANG,Angola
5,ANT,Antigua & Barbuda
6,ARG,Argentina
7,ARM,Armenia
8,ARU,Aruba
9,ASE,ASEAN (what's this?)


In [13]:
FIDE.to_csv(r"C:\Users\Anuj\Documents\GitHub\FIDE\Chess Scripts\Step 3 - FIDE Country Codes\Country Data\FIDE_codes.csv", 
            sep=',', encoding='utf-8', index = False)

### Cleaning

The purpose of this document is to clean the `.csv` files stored in the *Step 2 - Reformat/Data csvs/* folder.

Before beginning, we need to import several packages that help run the code below.


In [ ]:
library(knitr)
library(dplyr)
library(data.table)
library(lubridate)

### Access the data

In order to access the data we need to specify the folder we want to access the files from. We do this by defining a path to find them and set the path with `opts_knit$set(root.dir = path)`. In an R-Markdown document, it is required that you set the directory using this function. We can see the files in the directory with `head()`.

We also create a destination path with `dest` for later use so that we can export the files properly. `country_path` is a path to a dataset from which we will reference.

In [ ]:
path = "~/GitHub/FIDE/Chess Scripts/Step 2 - Reformat/Data csvs/"
country_path = "~/GitHub/FIDE/Chess Scripts/Step 3 - FIDE Country Codes/Country Data/FIDE_codes.csv"
dest =  "~/GitHub/FIDE/Chess Scripts/Step 4 - Cleaning/Cleaned csvs/"
opts_knit$set(root.dir = path)

temp = list.files(path = path)
full_path <- paste(path, temp, sep = "")

head(temp)

### Create functions to rename datasets

Below, I define a few functions that help us rename the datasets. 

`month` gets the first 3 letters of the temp elements, the month name
`num` converts each month into a number
`add_zero` converts each month number into a usable form when we create dates later on.

In [ ]:
month <- function(x){return(substr(x, 1, 3))}
num <- function(x) match(tolower(x), tolower(month.abb))
add_zero <- function(x){if (x <= 9){x = paste("0", x, sep = "")}; return(x)}

### Get the month number of all of the files in the dataset

Below, we rename create the variables names when they are assigned and imported.

In [ ]:
temp%>%
sapply(month)%>%
sapply(num)%>%
sapply(add_zero)%>%
paste("20", substr(temp, 4, 5), "-", ., "-", "01", sep = "")-> month_num

head(month_num)

We can see that the datasets correspond to dates on which the is recorded.

### Import all datasets

Below, we import and assign all of the datasets into memory. The only objects we need to hold onto is the data and several folder path references for later on. Therefore, we will remove everything unneeded with `rm(list=setdiff(ls())` below.

In [ ]:
for(i in 1:length(full_path)) {
assign(month_num[i], fread(full_path[i], sep = "*", data.table = FALSE, 
                           strip.white = TRUE, blank.lines.skip = TRUE))
}

FIDE <- mget(ls(pattern = "[0-9][0-9]-[0-9][0-9]"))

rm(list=setdiff(ls(), c("FIDE", "path", "temp", "dest", "country_path")))

### Data prep

In order to get the data to have useful and common values, we need to rename dozens of columns and values. 

In [ ]:
vector_months <- c(month.abb, tolower(month.abb),toupper(month.abb))
string = ""
for (i in 1:length(vector_months)){
if (i == 1){string = vector_months[i]}
else if (i > 1) {string = paste(string, vector_months[i], sep = "|")}
}
string = paste(string, "RATING", sep = "|")


new = c("CM", "WCM", "WCM", "WGM", "WFM", "WFM", "GM", "IM", "FM", "WIM", "GM")
old = c("c", "wc", "WC", "wg", "WF", "wf", "g", "m", "f", "wm", "gm" )

dates = as.Date(names(FIDE))
months_vec <- months(dates)%>%toupper()%>%substr(., 1, 3) 
year_vec <- year(dates)%>%as.character()%>%substr(3,4)
files <- paste(months_vec, year_vec, ".csv", sep = "")

codes <- fread(country_path, 
               sep =  ",", header = TRUE)


country_codes <- c("BDI", "BHU", "BUR", "CAF", "CAM", "CGO", "CIV", "CMR", "COD", 
                   "CPV", "CUR", "DJI", "ERI", "FID", "FIE", "GAB", "GUM", "Ind", 
                   "IVC", "KOR", "KOS", "KSA", "LAO", "LBN", "LBR", "LCA", "LES", 
                   "MDV", "MTN", "NET", "NRU", "OMA", "PLW", "ROU", "SCG", "SGP", 
                   "SLE", "SOL", "SSD", "STP", "SWZ", "TLS", "TPE", "TTO") 

countries <- c("Berundi", "Bhutan", "Burkina Faso", "Central African Republic", "Cambodia", 
            "Republic of the Congo", "Cote d'Ivoire", "Cameroon", "Democratic Congo", 
            "Cape Verde", "Curaçao", "Djibouti", "Eritrea", "Finland", "FIE", "Gabon", 
            "Guam", "India", "Côte d'Ivoire", "South Korea", "Kosovo", "Saudi Arabia", 
            "Laos", "Lebanon", "Liberia", "Saint Lucia", "Lesotho", "Maldives", "Mauritania", 
            "NET", "Nauru", "Oman", "Palau", "Romania", "Serbia and Montenegro", "Singapore", 
            "Sierra Leonne", "Solomon Islands", "South Sudan", "Sao Tome and Principe", 
            "Swaziland", "East Timor (Timor-Leste)	", "Taiwan", "Trinidad and Tobago") 

### Ugly data cleaning

This is the ugly part of the document: a `for loop` that is really meaty. Essentially, we will iterate all of the FIDE datasets to adjust each data's columns and values. We need to do this because want common values to merge the data later. 

You are free to use `data.table`'s `rbindlist()` function to merge all of the datasets within `FIDE`, but I chose not to in this file.

In [ ]:
for(i in 1:length(FIDE)){
  colnames(FIDE[[i]])[grepl("Name|NAME|name", colnames(FIDE[[i]]))] <- "Name"
  colnames(FIDE[[i]])[grepl("NUMBER", colnames(FIDE[[i]]))] <- "ID_Number"
  colnames(FIDE[[i]])[grepl("Fed|FED|COUNTRY", colnames(FIDE[[i]]))] <- "Country"
  colnames(FIDE[[i]])[grepl("Gms|GAMES|GM|Game|GAME", colnames(FIDE[[i]]))] <- "Games"
  colnames(FIDE[[i]])[grepl("K", colnames(FIDE[[i]]))] <- "K_factor"
  colnames(FIDE[[i]])[grepl("FLAG|Flag|flag", colnames(FIDE[[i]]))] <- "Activity"
  colnames(FIDE[[i]])[colnames(FIDE[[i]]) %in% c("Wtit","wtit","WTIT", "WTit")] <- "Womens_Title"
  colnames(FIDE[[i]])[colnames(FIDE[[i]]) %in% c("TITLE","Title","title","Tit")] <- "Title"
  colnames(FIDE[[i]])[grepl(string, colnames(FIDE[[i]]))] <- "Rating"
  colnames(FIDE[[i]])[grepl("Born|Age|age|BIRTHDAY|B-day|Bday", colnames(FIDE[[i]]))] <- "Age_Birthday"
  colnames(FIDE[[i]])[grepl("SEX", colnames(FIDE[[i]]))] <- "Sex"
  colnames(FIDE[[i]])[grepl("FOA", colnames(FIDE[[i]]))] <- "FIDE_Online_Arena"
  colnames(FIDE[[i]])[grepl("OTit", colnames(FIDE[[i]]))] <- "Other_Titles"
  FIDE[[i]] <- FIDE[[i]] %>%
               mutate(Date = as.POSIXct(names(FIDE)[i], format="%Y-%m-%d"),
                      Date_numeric = year(Date)+yday(Date)/366,
                      Rating = as.numeric(Rating),
                      Title= c(new, Title)[match(Title, c(old, Title))],
                      Country = c(codes$Country, Country)[match(Country, c(codes$Code, Country))],
                      Country = c(countries, Country)[match(Country, c(country_codes, Country))])%>%
                      filter(!Country %in% c("Fed", "Col"))%>%
                      select(-one_of("V1"))
  fwrite(FIDE[[i]] , file = paste(dest, files[i], sep = ""), sep = "*")
}

### Example data after modifications

In [ ]:
FIDE[[length(FIDE)]]%>%
head()%>%
select(Name, Country, Rating, Title, Date)%>%
kable()

In [ ]:
list.files(path = dest, pattern = "*.csv")%>%head()

# Analysis

In [ ]:
library(tidyverse)
library(data.table)
library(cowplot)
library(scales)
library(knitr)
library(zoo)
library(ggdark)
library(gganimate)
library(gg3D)
library(ggwordcloud)

In [ ]:
path = "~/GitHub/FIDE/Chess Scripts/Step 4 - Cleaning/Cleaned csvs/"
opts_knit$set(root.dir = path)
temp = list.files(path = path, pattern = "*.csv")
proper_temp <- paste(path, temp, sep = "")

num <- function(x) match(tolower(x), tolower(month.abb))
month <- function(x){return(substr(x, 1, 3))}
add_zero <- function(x){if (x <= 9){x = paste("0", x, sep = "")}; return(x)}

temp%>%
sapply(month)%>%
sapply(num)%>%
sapply(add_zero)%>%
paste("20", substr(temp, 4, 5), "-", ., "-", "01", sep = "")-> month_num


for(i in 1:length(proper_temp)) {
assign(month_num[i], fread(proper_temp[i], sep = "*", data.table = FALSE, 
                           strip.white = TRUE, blank.lines.skip = TRUE,
                           nThread= parallel::detectCores()))
}

#statement below takes a very long time to run because of rbindlist()
FIDE <- mget(ls(pattern = "[0-9][0-9]-[0-9][0-9]"))%>%
        rbindlist(fill = TRUE)
        
rm(list=setdiff(ls(), c("FIDE")))

In [ ]:
regular_titles = c("CM", "FM", "IM", "GM")
Titles = c("CM", "WCM",  "WGM", "WFM", "IM", "FM", "WIM", "GM", "")
Titles_only = c("CM", "WCM",  "WGM", "WFM", "IM", "FM", "WIM", "GM")
Active = c("", "w")
Inactive = c("i", "wi")


Active_player <- FIDE%>%
                 filter(Activity %in% Active)%>%
                 group_by(Date_numeric)%>%
                 summarise(total_count = n(),
                           avg_rating = mean(Rating, na.rm = T),
                           sd_rating = sd(Rating, na.rm = T))%>%
                 mutate(gain_loss = c(NA, diff(total_count)))


Inactive_player <- FIDE%>%
                   filter(Activity %in% Inactive)%>%
                   group_by(Date_numeric)%>%
                   summarise(total_count = n(),
                             avg_rating = mean(Rating, na.rm = T),
                             sd_rating = sd(Rating, na.rm = T))%>%
                   mutate(gain_loss = c(NA, diff(total_count)))

All_players <- FIDE%>%
               filter(Activity %in% c(Active, Inactive))%>%
               group_by(Date_numeric)%>%
               summarise(total_count = n(),
               avg_rating = mean(Rating, na.rm = T),
               sd_rating = sd(Rating, na.rm = T))%>%
               mutate(gain_loss = c(NA, diff(total_count)))



All_countries <- FIDE%>%
                 filter(Activity %in% c(Active, Inactive))%>%
                 group_by(Date_numeric, Country)%>%
                 summarise(total_count = n())

Country_diff <- All_countries%>%
                arrange(Country, Date_numeric)%>%
                group_by(Country)%>%
                mutate(Diff = total_count - lag(total_count))%>%
                filter(!is.na(Diff))


Notable <- Country_diff%>%
           filter(abs(Diff) > 580)%>%
           .$Country%>%
           unique()

w_old = c("WCM", "WFM", "WGM", "WIM")
w_new = rep("W", length(w_old))

FIDE%>%
filter(Title %in% Titles_only, Activity%in% Active)%>%
group_by(Date_numeric, Title)%>%
tally()%>%
mutate(w_titles = c(w_new, Title)[match(Title, c(w_old, Title))])-> titled

titled%>%
group_by(Date_numeric, w_titles)%>%
summarise(n = sum(n)) -> titles_group

FIDE%>%
filter(Games > 0, Activity %in% Active)%>%
group_by(Date_numeric)%>%
summarise(n = n())%>%
mutate(roll = rollmean(n, k = 6, fill = NA))-> Games


strongest <- FIDE%>%
             filter(Date_numeric == max(Date_numeric), 
                    Activity %in% Active,
                    as.numeric(Age_Birthday) > 1900, 
                    as.numeric(Age_Birthday) < 2019,
                    Rating > 900, 
                    Rating < 3000)%>%
             transmute(Country, Age = 2020-as.numeric(Age_Birthday), Rating)


breaks <- c(6, 19, 30, 41, 53, 65, 96)
break_labels = paste(breaks[-length(breaks)], breaks[-1], sep = "-")

strongest <- strongest%>%
             mutate(Age_groups = cut(Age, 
                                    breaks=breaks, 
                                    include.lowest=TRUE, 
                                    right=FALSE, 
                                    labels=break_labels))%>%
             filter(!is.na(Age_groups))


country_grouped = strongest%>%
                  group_by(Country, Age_groups)%>%
                  summarise(count = n(), average_rating = mean(Rating)%>%round())%>%
                  filter(count > 100)

country_grouped%>%
    group_by(Country)%>%
    filter(n() == 6)%>%
    arrange(Age_groups, -average_rating)%>%
    group_by(Age_groups)%>%
    top_n(n = 10)%>%
    .$Country%>%
    unique()-> country_vec

country_grouped%>%
filter(Country %in% country_vec)%>%
select(-count) -> strongest

titled_only <- FIDE%>%
               filter(Title %in% c("CM", "FM", "IM", "GM"), Date_numeric == max(Date_numeric))%>%
               group_by(Title, Country)%>%
               tally()%>%
               filter(n > 15)%>%
               group_by(Country)%>%
               filter(n() == 4)


dec19_titled <- FIDE%>%
                filter(Date_numeric == max(Date_numeric), 
                      Activity %in% Active)%>%
                select(Rating, Title, Age_Birthday, Womens_Title)%>%
                mutate(Age_Birthday = 2020 - as.numeric(Age_Birthday))%>%
                filter(Age_Birthday > 0, Age_Birthday < 120,
                      Rating > 950, !Title %in% c(""), Womens_Title == "")


hist_data <- FIDE%>%
             filter(Date_numeric == max(Date_numeric), 
                    Activity %in% Active,
                    as.numeric(Age_Birthday) > 1900)%>%
             select(Sex, Rating)
             

Name <- FIDE%>%
        filter(Date_numeric %in% max(Date_numeric), 
               Activity %in% c("w", ""),
               grepl(",", Name, fixed=TRUE))%>%
        select(Name)%>%
        mutate(comma = str_count(Name, ","))%>%
        filter(comma == 1)%>%
        select(-comma)%>%
        separate(Name, c("Last", "First"), ",")%>%
        gather("Type", "Name")%>%
        group_by(Type, Name)%>%
        summarise(n = n())%>%
        arrange(Type, -n)%>%
        group_by(Type)%>%
        top_n(n = 30)

#write dataframe to folder

# DO NOT RUN UNLESS YOU WANT THE BIG file

# fwrite(FIDE, "FIDE.csv")
# FIDE <- fread("FIDE.csv", data.table = FALSE)



# all_titled <- FIDE%>%
#               filter(Activity %in% Active,
#                      Rating > 960, Rating < 2900, Title %in% c("FM", "IM", "GM"))%>%
#               select(Title,  Rating, Date_numeric, Age_Birthday)%>%
#               mutate(year = as.numeric(Age_Birthday))%>%
#               filter(year > 1900, year < 2020)%>%
#               mutate(Age = Date_numeric-year)%>%
#               select(-Age_Birthday, -year)%>%
#               na.omit()


In [ ]:
ggplot(data = All_players, aes(x = Date_numeric, y = total_count)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of players")+
  dark_theme_gray()+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p1

ggplot(data = Active_player, aes(x = Date_numeric, y = total_count)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of actives")+
  # geom_segment(aes(x=as.Date("2004-06-01"), xend=as.Date("2008-06-01"), y=120000, yend=70000), 
  #              arrow = arrow(length = unit(.5, "cm")), size = 2)+
  # geom_segment(aes(x=as.Date("2004-06-01"), xend=as.Date("2002-02-01"), y=120000, yend=35000), 
  #              arrow = arrow(length = unit(.5, "cm")), size = 2)+ 
  # annotate("text", x = as.Date("2006-06-01"), y = 130000, 
  #          label = "Economic crash of 2008 (the larger chasm)", size = 5) +
  dark_theme_gray()+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01)) -> p2

ggplot(data = Inactive_player, aes(x = Date_numeric, y = total_count)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of inactives")+
  # geom_segment(aes(x=as.Date("2004-06-01"), xend=as.Date("2008-06-01"), y=140000, yend=55000), 
  #              arrow = arrow(length = unit(.5, "cm")), size = 2)+
  # geom_segment(aes(x=as.Date("2004-06-01"), xend=as.Date("2002-07-01"), y=140000, yend=35000), 
  #              arrow = arrow(length = unit(.5, "cm")), size = 2)+
  # annotate("text", x = as.Date("2005-01-01"), y = 155000, 
  #          label = "Is this corrupted data?", size = 5)+
  dark_theme_gray()+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p3

ggplot(data = All_players, aes(x = Date_numeric, y = gain_loss)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of gained/lost")+
  ggtitle("# of total gained/lost")+
  geom_line(aes(y = 0))+
  dark_theme_gray()+
  theme(plot.title = element_text(size = 6, face = "bold"))+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p4

ggplot(data = Active_player, aes(x = Date_numeric, y = gain_loss)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of gained/lost")+  
  ggtitle("# of Actives gained/lost")+
  geom_line(aes(y = 0))+
  dark_theme_gray()+
  theme(plot.title = element_text(size = 6, face = "bold"))+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p5

ggplot(data = Inactive_player, aes(x = Date_numeric, y = gain_loss)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("# of gained/lost")+
  ggtitle("# of Inactives gained/lost")+
  geom_line(aes(y = 0))+
  dark_theme_gray()+
  theme(plot.title = element_text(size = 6, face = "bold"))+
  scale_y_continuous(labels = comma)+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p6

ggplot(data = Active_player, aes(x = Date_numeric, y = avg_rating)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("Average of Actives")+
  dark_theme_gray()+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p7

ggplot(data = Inactive_player, aes(x = Date_numeric, y = avg_rating)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("Average of Inactives")+
  dark_theme_gray()+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p8

ggplot(data = Active_player, aes(x = Date_numeric, y = sd_rating)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("Actives SD")+
  dark_theme_gray()+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p9

ggplot(data = Inactive_player, aes(x = Date_numeric, y = sd_rating)) + 
  geom_line(color = "#FC4E07", size = 1)+
  xlab("Year")+
  ylab("Inactives SD")+
  dark_theme_gray()+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p10

Country_diff%>%
filter(Country %in% Notable)%>%
ggplot()+
geom_line(aes(x = Date_numeric, y = Diff, colour = Country))+
dark_theme_classic()+
facet_wrap(~Country)+
ggtitle("Gained/Lost over time")+
xlab("Year")+
ylab("Gained/Lost")+ 
theme(legend.position = "none")+
geom_hline(aes(yintercept = 0))+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .05), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .05))-> p11

titled%>%
ggplot(aes(x=Date_numeric, y= n, group=Title, color=Title)) +
geom_line(size = 1.5)+
dark_theme_gray()+
xlab("Year")+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01))-> p12

titles_group%>%
ggplot(aes(x=Date_numeric, y= n, group=w_titles, color=w_titles)) +
geom_line(size = 1.5)+
dark_theme_gray()+
xlab("Year")+
ylab("# of players")+
labs(color="")+
theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
      panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01)) -> p13

titles_group %>%
group_by(w_titles) %>%
mutate(Diff = n - lag(n))%>%
filter(w_titles != "W")%>%
ggplot(aes(x=Date_numeric, y= Diff, group=w_titles, color=w_titles)) +
geom_line(size = 1)+
labs(x = "Year", y = "Gained/Lost", color="")+
dark_theme_gray()+
facet_wrap(~w_titles, ncol  = 2, scales = "free")+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01),
        legend.position = "none")-> p14

Games%>%
ggplot() +
geom_line(aes(x=Date_numeric, y= roll), size = 1.5, color = "red")+
dark_theme_classic()+
xlab("Year")+
theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
      panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01)) -> p15

titled_only%>%
filter(Title %in% c("CM", "FM", "IM", "GM"))%>%
ggplot(aes(x= Country, y =n, fill = Country))+
    geom_bar(stat ="identity")+
    coord_flip()+
    facet_wrap(~Title, scales = "free")+
    dark_theme_gray()+
    xlab("")+ylab("")+
    theme(axis.text=element_text(size=6))+
    theme(panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank(),
          panel.background = element_blank())+
    theme(legend.position = "none")-> p16


strongest%>%
ggplot(aes(x= Country, y= average_rating, fill = Country))+
    geom_bar(stat ="identity")+
    coord_flip()+
    ylab("Average Rating")+
    ggtitle("Highest rated age groups by country")+
    facet_wrap(~Age_groups, ncol = 2, scales="free")+
    dark_theme_gray()+
    theme(axis.text=element_text(size=6))+
    theme(legend.position = "none",
          panel.background = element_blank())-> p17

dec19_titled%>%
ggplot(aes(Age_Birthday, Rating, color=Title)) +
  geom_point(alpha = .3) +
  scale_fill_brewer(palette="Set1") +
  dark_theme_classic()+
  theme(legend.position = "bottom")+
  xlab("Age")+
  ylab("Rating")+
  theme(panel.grid.major = element_line(linetype = 'solid',colour = "white", size = .01), 
        panel.grid.minor = element_line(linetype = 'solid',colour = "white", size = .01),
        panel.background = element_blank()) -> p18


hist_data%>%
mutate(Sex = recode_factor(Sex, "M" = "Male", "F" = "Female"))%>%
magrittr::set_colnames(c("Male or Female?", "Rating"))%>%
ggplot(aes(Rating, fill = `Male or Female?`, colour = `Male or Female?`)) +
geom_density(alpha = .6)+
labs(x = "", y = "")+
dark_theme_classic()+
theme(panel.background = element_blank(),
      axis.text.y = element_blank(),
      axis.ticks.y = element_blank(),
      legend.position = c(.8, .6)) -> p19

Name%>%
filter(Type == "First")%>%
ggplot(aes(label = Name, size = n, color = Name)) +
  geom_text_wordcloud() +
  dark_theme_minimal()+
  scale_size_area(max_size = 15) -> p20

Name%>%
filter(Type == "Last")%>%
ggplot(aes(label = Name, size = n, color = Name)) +
  geom_text_wordcloud() +
  dark_theme_minimal()+
  scale_size_area(max_size = 15) -> p21

# dec19_titled%>%
# ggplot(aes(x = Age_Birthday, y = Rating, fill = Title)) +
#   stat_density_2d(geom = "polygon", 
#                   aes(alpha = ..level..),
#                   bins = 3)+
#   dark_theme_classic()

# find_hull <- function(df) df[chull(df$Age, df$Rating), ]
# hulls <-plyr::ddply(all_titled, "Title", find_hull)
# 
# all_titled%>%
# filter(Date_numeric == max(Date_numeric))%>%
# ggplot(aes(Age, Rating, color=Title, fill = Title)) +
# geom_point(alpha = .3) +
# geom_polygon(data = hulls, alpha = 0.5) +
# scale_fill_brewer(palette="Set1") +
# theme(legend.position = "right")+
# ggtitle("Age vs Rating")+
# xlab("Age")+
# ylab("Rating")
# + labs(title = 'Year: {frame_time}')
# +transition_time(Date_numeric)


The purpose of this document is to visually analyze all of the FIDE data files collected in the previous step's folder. 

### Irregular values by year


In [ ]:
FIDE%>%
filter(!Title %in% Titles | !Activity %in% c(Active, Inactive))%>%
group_by(Date_numeric)%>%
tally()%>%
arrange(-n)%>%
head(10)%>%
magrittr::set_colnames(c("Date", "# of irregular values"))%>%
kable()

As we can see from the table above, most of irregular values in the files come from early on (2001 - 2005) rather than the latest files.

I'll look to address many of the values in the early datasets eventually. For now though, over 99.9% of the data is interpretable. 

\pagebreak


### Total player count over time

In [ ]:
plot_grid(p1, p4, p2, p5, p3, p6, labels = c("A", "B", "C", "D", "E", "F"), ncol = 2) 

The four charts above reveal the total count of FIDE's members over time. Chart `A` shows a smooth gradual increasing growth curve among total players. This may lead you to believe that more chess players are playing tournaments, but a player's **activity** is a better metric to go by. 

**Activity** is defined by if a given FIDE player had played a rated FIDE game within the past 12 months. If we take this into account, charts `C` and `E` show how the total active & inactive player count increase over time. Both charts show a fairly linear trend over time, but from 2007 to 2010 in each graph, there was a noticeable drop off in the active player base and increase in the inactive player base. I have my doubts on if there is faulty data here because chart `A` shows no irregularity during that time. This dip may be due to the economic crash during that time, but I need to do more exploration on this topic before making any definitive statements. Exploring variation by country may also be worth doing.

Charts `B`, `D` and `F` show the number of players gained and lost over time. The most relevant of the 3 graphs is `B` which shows several instances where total player counts dropped off. I genuinely don't if the data is faulty because of my doing or if FIDE is providing incomplete data sets based off of the charts.

\pagebreak

### Rating stability over time (misleading)

In [ ]:
plot_grid(p7, p8, p9, p10, labels = c("G", "H", "I", "J")) 

Each chart above shows how the Active and Inactive player's average rating and rating standard deviation have progressed over time. For the most part, it is a meaningless metric because FIDE has brought in more **lower** rated chess players into the player pool over time. 

This explains the steady decrease in average rating and increase in standard deviation over time.

\pagebreak


### Which countries have seen the greatest changes in player counts?

In [ ]:
p11

A consistent theme we will see in every graph is a potential corruption of data. The graph above displays the of players gained and lost over time by country. 

- Bulgaria, Finland, Germany, Serbia and Montenegro, and the United State of America saw unusual spikes & dips at various junctures (2002, 2007, 2017)

\pagebreak

### Titled player count over time

In [ ]:
p13

The plot above shows a few interesting trends (W = All women's titles):

- All titled player groups have significantly increased in size over the past 20 years

- Around both 2002 and 2007, there were significant dips in all categories, except in `GM`s and `CM`s. 

- There are about as many `GM`s as `CM`s. It goes to show how stigmatized the CM title is.

\pagebreak

### Titled players gained/lost over time

In [ ]:
p14

The graph above is graphs the rolling difference of titled player counts over time. As we saw before, there are a few noticeable dips in the graph. Most notably they occur around 2002, 2008 and 2010 in varying degrees. 

\pagebreak

Notable observations:

- Clearly, among `FM`, `IM` and `GM` players, Russia has the most titled players by a large margin. This is not surprising because of Russia's longstanding history (USSR and Soviet Union) of chess.

- Germany is a clear 2nd amongst `FM`, `IM` and `GM` players. They also lead the pack in `CM` players. This is surprising to me since I've never thought of Germany as being a bastion of strong players. 

- Every other country lags behind these top-tier powerhouses.

\pagebreak

### Strongest countries by age group (Work in progress)

### Titled player counts by country

In [ ]:
p17 

This plot would be substantially better at showing relative strengths among countries if I could scale it differently. This will adjusted in future iterations, probably through scaling.

\pagebreak

### Age vs Rating of titled players (December 2019)

In [ ]:
p18 

The most aesthetically pleasing graph to look at is the one above. It reveals a few aesthetically pleasing observations:

- Bands of players can be separated by titles categories: an obvious points is that the top blue band is all GMs, the highest rated group. Below the  blue band are IMs (purple band), FMs (green band) and CMs (red band). 

- CMs and FMs vary greatly across rating categories because it has become much easier for lower rated players to acquire titles in youth tournaments and via inter zonal tournaments.

- There is a slight negative correlation, among all titled player groups, between Age and Rating. 

### Males/Females rating distributions (December 2019)

In [ ]:
p19

Above is a plot showing the stark difference between male and female rating distributions for the previous month. I'll look to explore why the female rating distribution is sharply skewed right compared to the male counterpart. 

### Word Cloud: First names

In [ ]:
p20

### Word Cloud: Last names

In [ ]:
p21

In [ ]:

# FIDE%>%
# filter(Rating > 1000, Rating < 2900)%>%
# group_by(Date_numeric)%>% 
# mutate(percrank=rank(Rating)/length(Rating))%>%
# filter(Name == "Dahiya, Anuj") -> Anuj

# library(xts)
# library(dygraphs)
# library(foreach)
# library(doParallel)
# library(stringi)
# library(lubridate)

# library(ggplot2)
# library(gganimate)
# library(tidyverse)
# 
# fill_in = data.frame(value = c(), year = c())
# for (i in seq(1, 300, by = 10)){
#   value = rnorm(i*20)
#   year = rep(i, length(value))+2000
#   stuff = data.frame(value = value, year = year)
#   fill_in = rbind(fill_in, stuff)
# }
# 
# 
# ggplot(data = fill_in, mapping = aes(x=value, fill=as.factor(year)))+ 
# geom_density(alpha = .3)
# 
# 
# 
# ggplot(fill_in, aes(value)) +
# geom_density(alpha = .3)+
# labs(title = 'Year: {frame_time}') +
# transition_time(year)




# FIDE%>%
# filter(Rating > 2830)%>%
# .$Name%>%
# unique() -> top_player
# 
# #Visualize the data
# WC_caliber_players <- FIDE%>%
#   filter(Name %in% top_player, Activity == "")%>%
#   select(Name, Rating, Date)%>%
#   arrange(Name, Date)
# 
# 
# dygraphed <- WC_caliber_players %>%
#              spread(key = Name, value = Rating)%>%
#              xts(.[,which(colnames(.)!= "Date")],order.by =  .$Date)
# 
# dygraph(dygraphed, main = "Player's rating over Time") %>%
#   dyOptions(drawPoints = TRUE, pointSize = 2, axisLineWidth = 4) %>%
#   dyAxis("y", label = "Rating", valueRange = c(1900, 3200))%>%
#   dyRangeSelector()%>%
#   dyLegend(width = 400)
#
# data <- fread("SEP19.csv", sep = "*", data.table = FALSE)
#
#
# library(ggplot2)
# data%>%
# mutate(Rating = as.numeric(NOV19))%>%
# filter(Flag == "wi")%>%
# na.omit()%>%
# ggplot(aes_string(x="Rating"))+
# geom_density(size=2, alpha=.4)+
# geom_histogram(aes(y = ..density..), bins = 50, col= "red")+ 
# labs(title="Histogram overlayed with Density curve") +
# labs(x="Rating", y="Percentage of players in population")


# years_increment = FIDE$Date_numeric%>%
#                   unique()%>%
#                  .[seq(1, length(.), 1)]
# 
# all_titled <- FIDE%>%
#               filter(Activity %in% Active,
#                      Rating > 960, Rating < 2900, 
#                      Title %in% c("FM", "IM", "GM"))%>%
#               select(Title,  Rating, 
#                      Date_numeric, Age_Birthday)%>%
#               mutate(year = as.numeric(Age_Birthday))%>%
#               filter(year > 1900, year < 2020)%>%
#               mutate(Age = Date_numeric-year)%>%
#               select(-Age_Birthday, -year)%>%
#               na.omit()%>%
#               filter(Date_numeric %in% years_increment)



# find_hull <- function(df) df[chull(df$Age, df$Rating), ]
# hulls <-plyr::ddply(all_titled, "Title", find_hull)

# all_titled%>%
# ggplot(aes(Age, Rating, color=Title))+ # fill = Title here
# geom_point(alpha = .3)+
# # geom_polygon(data = hulls, alpha = 0.5) +
# dark_theme_bw()+
# scale_fill_brewer(palette="Set1")+
# theme(legend.position = "bottom")+
# ggtitle("Age vs Rating")+
# xlab("Age")+
# ylab("Rating")+
# labs(title = 'Year: {frame_time}')+
# transition_time(Date_numeric)
# 
# 
# ggplot(iris, aes(Sepal.Width, Petal.Width)) +
# geom_point() +
# labs(title = "{closest_state}") +
# transition_states(Species, transition_length = 3, state_length = 1)



# FIDE%>%
# filter(Activity %in% Active, Title %in% regular_titles)%>%
# transmute(Title, Rating, 
#           Date = Date_numeric,
#           Age = Date - as.numeric(Age_Birthday))%>%
# filter(Rating > 999, Rating < 2900, Age > 0, Age < 200) -> exp
# 
# random = sample.int(nrow(exp), 1000)
# 
# 
# exp[random,]%>%
# ggplot(aes(x=Rating, y=Age, z=Date, color=Title)) + 
# dark_theme_void() +
# axes_3D() +
# stat_3D(theta = 10)+
# labs_3D(labs=c("Rating", "Age", "Date"))

# best_titled <- FIDE%>%
#                filter(Title %in% c("CM", "FM", "IM", "GM"))%>%
#                group_by(Title, Country, Date_numeric)%>%
#                tally()%>%
#                filter(n > 15)%>%
#                group_by(Country)%>%
#                filter(n() >= 400)%>%
#                group_by(Country)%>%
#                tally()%>%
#                arrange(-n)%>%
#                head(10)%>%
#                .$Country
# 
# FIDE%>%
# filter(Title %in% regular_titles, Country %in% best_titled)%>%
# group_by(Title, Country, Date_numeric, .drop = FALSE)%>%
# tally()%>%
# filter(Date_numeric == max(Date_numeric))%>%
# ggplot(aes(x= Country, y =n, fill = Country))+
#     geom_bar(stat ="identity")+
#     coord_flip()+
#     facet_wrap(~Title, scales = "free")+
#     dark_theme_gray()+
#     xlab("")+ylab("")+
#     theme(axis.text=element_text(size=6))+
#     theme(panel.grid.major = element_blank(), 
#           panel.grid.minor = element_blank(),
#           panel.background = element_blank())+
#     theme(legend.position = "none")-> p16

# FIDE%>%
# filter(Name %in% c("Carlsen, Magnus", 
#                     "Rawski, Slawomir", 
#                     "Anand, Viswanathan"))%>%
# select(Name, Rating, Date_numeric)%>%
# ggplot(aes(x = Date_numeric, y = Rating, colour = Name))+
# geom_line(size=1.5)+
# dark_theme_bw()+
# theme(legend.position = "bottom", 
#       plot.title = element_text(hjust = 0.5),
#       panel.background = element_blank())+
# xlab("Year")+
# ylab("Rating")+
# ggtitle("Rating over time")+
# guides(color=guide_legend(""))


# FIDE%>%
# group_by(Name)%>%
# filter(n() >= 138)%>%
# filter(Rating > 999, Rating < 2900, 
#        Date_numeric %in% c(max(Date_numeric), min(Date_numeric)))%>%
# group_by(Name)%>%
# mutate(diff = Rating - lag(Rating))%>%
# arrange(diff)%>%
# filter(Activity %in% c("", "w"))-> maxed



# library(forcats)
# library(ggridges)
# 
# players <- FIDE%>%
#            filter(Activity %in% c("w", ""),
#                   Rating > 1000, Rating < 2900,
#                   Sex %in% c("M", "F"))%>%
#            select(Rating, Date_numeric, Sex)%>% 
#            group_by(Date_numeric)%>% 
#            sample_n(1000)%>%
#            ungroup()
# 
#            
# 
# players %>%
# mutate(Date_numeric = fct_rev(as.factor(Date_numeric%>%round()))) %>%
# ggplot(aes(y = Date_numeric)) +
# geom_density_ridges(aes(x = Rating, fill = paste(Date_numeric, Sex)), 
#                     alpha = .8, color = "white"
#                     # , from = 0, to = 100
#                     ) +
# # labs(x = "Vote (%)", y = "Election Year", title = "Indy vs Unionist vote in Catalan elections",
#      # subtitle = "Analysis unit: municipalities (n = 949)",
#      # caption = "Marc Belzunces (@marcbeldata) | Source: Idescat") +
# # scale_y_discrete(expand = c(0, 0)) +
# # scale_x_continuous(expand = c(0, 0)) +
# scale_fill_cyclical(breaks = c("Male", "Female"),
#                     labels = c(`Male` = "Nale", `Female` = "Female"),
#                     values = c("#ff0000", "#0000ff", "#ff8080", "#8080ff"),
#                     name = "Sex", guide = "legend")+
# coord_cartesian(clip = "off") +
# theme_ridges(grid = FALSE)+
# dark_theme_classic()